# Reshape Data in Polars Efficiently from Wide to Long Form - Part II

## **Introduction**

In a [previous blogpost](Reshape-Data-in-Polars-Wide-to_Long-Part-I.ipynb), we explored various scenarios when reshaping a DataFrame from wide to long form.

This article focuses on reshaping efficiently, especially for large datasets, where some post processing is done after [unpivoting](https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.unpivot.html#polars.DataFrame.unpivot). 

For the most part, we will reuse the datasets used in the [previous blogpost](Reshape-Data-in-Polars-Wide-to_Long-Part-I.ipynb).

Polars' [eager API](https://docs.pola.rs/api/python/stable/reference/dataframe/index.html) is used in the examples below; for more performance, or in production mode, it is recommended to use the [lazy API](https://docs.pola.rs/api/python/stable/reference/lazyframe/index.html).

The examples are broken down in a line by line approach, instead of a chainable form, for illustration purposes - this way we can clearly identify what sections of the code have the biggest impact on performance.

Caveat: Do not optimise prematurely, or rather optimise only if you need to.

In [35]:
import polars as pl
import sys

print("polars version :", pl.__version__)
print("python version :", sys.version)

polars version : 1.7.1
python version : 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:51:49) [Clang 16.0.6 ]


(Many-variables-in-column-names)=
## **Many variables in column names**

In [36]:
who = pl.read_csv("./Data_files/who.csv", null_values="NA")
who

country,iso2,iso3,year,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,new_sp_m65,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sn_m014,new_sn_m1524,new_sn_m2534,new_sn_m3544,new_sn_m4554,new_sn_m5564,new_sn_m65,new_sn_f014,new_sn_f1524,new_sn_f2534,new_sn_f3544,new_sn_f4554,new_sn_f5564,new_sn_f65,new_ep_m014,new_ep_m1524,new_ep_m2534,new_ep_m3544,new_ep_m4554,new_ep_m5564,new_ep_m65,new_ep_f014,new_ep_f1524,new_ep_f2534,new_ep_f3544,new_ep_f4554,new_ep_f5564,new_ep_f65,newrel_m014,newrel_m1524,newrel_m2534,newrel_m3544,newrel_m4554,newrel_m5564,newrel_m65,newrel_f014,newrel_f1524,newrel_f2534,newrel_f3544,newrel_f4554,newrel_f5564,newrel_f65
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Afghanistan""","""AF""","""AFG""",1980,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Afghanistan""","""AF""","""AFG""",1981,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Afghanistan""","""AF""","""AFG""",1982,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Afghanistan""","""AF""","""AFG""",1983,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Afghanistan""","""AF""","""AFG""",1984,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zimbabwe""","""ZW""","""ZWE""",2009,125,578,null,3471,681,293,192,180,873,null,3028,419,229,126,1560,860,null,6496,1655,882,861,1425,1334,null,7023,1551,729,514,244,266,0,1922,491,231,223,210,394,0,1944,438,182,138,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Zimbabwe""","""ZW""","""ZWE""",2010,150,710,2208,1682,761,350,252,173,974,2185,1283,490,265,171,1826,821,3342,3270,1545,882,864,1732,1282,4013,2851,1377,789,563,270,243,902,868,418,229,192,220,319,1058,677,338,181,146,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Zimbabwe""","""ZW""","""ZWE""",2011,152,784,2467,2071,780,377,278,174,1084,2161,1386,448,274,160,1364,596,2473,2813,1264,702,728,1271,947,2754,2216,962,587,495,250,195,746,796,342,172,172,209,318,802,640,284,137,129,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In the [previous blogpost](Reshape-Data-in-Polars-Wide-to_Long-Part-I.ipynb), we flipped to long form using the code below:

In [37]:
regex = r"new_?(?<diagnosis>.*)_(?<gender>.)(?<age>.*)"
expression = pl.col("variable").str.extract_groups(regex)
out_naive = (
    who.unpivot(
        index=["country", "iso2", "iso3", "year"],
        value_name="count",
    )
    .with_columns(variable=expression)
    .unnest("variable")
    .cast({"count": pl.Int32})
)

out_naive

country,iso2,iso3,year,diagnosis,gender,age,count
str,str,str,i64,str,str,str,i32
"""Afghanistan""","""AF""","""AFG""",1980,"""sp""","""m""","""014""",null
"""Afghanistan""","""AF""","""AFG""",1981,"""sp""","""m""","""014""",null
"""Afghanistan""","""AF""","""AFG""",1982,"""sp""","""m""","""014""",null
"""Afghanistan""","""AF""","""AFG""",1983,"""sp""","""m""","""014""",null
"""Afghanistan""","""AF""","""AFG""",1984,"""sp""","""m""","""014""",null
…,…,…,…,…,…,…,…
"""Zimbabwe""","""ZW""","""ZWE""",2009,"""rel""","""f""","""65""",null
"""Zimbabwe""","""ZW""","""ZWE""",2010,"""rel""","""f""","""65""",null
"""Zimbabwe""","""ZW""","""ZWE""",2011,"""rel""","""f""","""65""",null


In [38]:
%%timeit
regex = r"new_?(?<diagnosis>.*)_(?<gender>.)(?<age>.*)"
expression = pl.col("variable").str.extract_groups(regex)
(
    who.unpivot(
        index = ["country", "iso2", "iso3", "year"],
        value_name="count",
    )
    .with_columns(variable=expression)
    .unnest("variable")
)

101 ms ± 230 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Is this good enough, or can we do better? Let's investigate with [lprun](https://github.com/pyutils/line_profiler):

In [39]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [40]:
def wide_to_long(df):
    # the computation is broken down
    # into parts for more clarity
    # on what happens at each stage
    out = df.unpivot(
        index=["country", "iso2", "iso3", "year"],
        value_name="count",
    )
    string = out.get_column("variable")
    regex = r"new_?(?<diagnosis>.*)_(?<gender>.)(?<age>.*)"
    string = string.str.extract_groups(regex)
    out = out.with_columns(variable=string)
    out = out.unnest("variable")
    return out

In [41]:
%lprun -f wide_to_long wide_to_long(df=who)

Timer unit: 1e-09 s

Total time: 0.103107 s
File: /var/folders/yh/003fk4jn5fl9kl3mnm1y0b1h0000gr/T/ipykernel_83318/3562139280.py
Function: wide_to_long at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def wide_to_long(df):
     2                                               # the computation is broken down
     3                                               # into parts for more clarity
     4                                               # on what happens at each stage
     5         2    3527000.0    2e+06      3.4      out = df.unpivot(
     6         1          0.0      0.0      0.0          index=["country", "iso2", "iso3", "year"],
     7         1          0.0      0.0      0.0          value_name="count",
     8                                               )
     9         1      11000.0  11000.0      0.0      string = out.get_column("variable")
    10         1       1000.0   1000.0      0.0      rege

```
Timer unit: 1e-09 s

Total time: 0.100968 s
File: /var/folders/yh/003fk4jn5fl9kl3mnm1y0b1h0000gr/T/ipykernel_83318/3562139280.py
Function: wide_to_long at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def wide_to_long(df):
     2                                               # the computation is broken down
     3                                               # into parts for more clarity
     4                                               # on what happens at each stage
     5         2    3317000.0    2e+06      3.3      out = df.unpivot(
     6         1          0.0      0.0      0.0          index=["country", "iso2", "iso3", "year"],
     7         1          0.0      0.0      0.0          value_name="count",
     8                                               )
     9         1      12000.0  12000.0      0.0      string = out.get_column("variable")
    10         1          0.0      0.0      0.0      regex = r"new_?(?<diagnosis>.*)_(?<gender>.)(?<age>.*)"
    11         1   97543000.0    1e+08     96.6      string = string.str.extract_groups(regex)
    12         1      82000.0  82000.0      0.1      out = out.with_columns(variable=string)
    13         1      14000.0  14000.0      0.0      out = out.unnest("variable")
    14         1          0.0      0.0      0.0      return out
```

The bulk of time is spent in the string computation, specifically the section where we extract the `diagnosis`, `gender` and `age` columns, using the [extract_groups](https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.str.extract_groups.html) method. 


Can we reduce this cost? How? What if we compute the string regex operation on the columns before unpivoting? 	🤔

In [42]:
index = ["country", "iso2", "iso3", "year"]
columns = pl.Series(who.columns)
mask = columns.is_in(index).not_()
columns = columns.filter(mask)
regex = r"new_?(?<diagnosis>.*)_(?<gender>.)(?<age>.*)"
columns = columns.str.extract_groups(regex)
columns = columns.struct.unnest()
columns

diagnosis,gender,age
str,str,str
"""sp""","""m""","""014"""
"""sp""","""m""","""1524"""
"""sp""","""m""","""2534"""
"""sp""","""m""","""3544"""
"""sp""","""m""","""4554"""
…,…,…
"""rel""","""f""","""2534"""
"""rel""","""f""","""3544"""
"""rel""","""f""","""4554"""


Next, we need to properly pair the extracted column with the unpivoted data: 

In [43]:
# implosion and unpivoting ensures we get a final length
# that is the same as the columns' length
# ensuring we can pair correctly
out = who.select(pl.implode("*"))
out = out.unpivot(index=index, value_name="count")
out = out.drop("variable")
out = out.with_columns(columns)
index.append("count")
out = out.explode(index)
out

country,iso2,iso3,year,count,diagnosis,gender,age
str,str,str,i64,i64,str,str,str
"""Afghanistan""","""AF""","""AFG""",1980,null,"""sp""","""m""","""014"""
"""Afghanistan""","""AF""","""AFG""",1981,null,"""sp""","""m""","""014"""
"""Afghanistan""","""AF""","""AFG""",1982,null,"""sp""","""m""","""014"""
"""Afghanistan""","""AF""","""AFG""",1983,null,"""sp""","""m""","""014"""
"""Afghanistan""","""AF""","""AFG""",1984,null,"""sp""","""m""","""014"""
…,…,…,…,…,…,…,…
"""Zimbabwe""","""ZW""","""ZWE""",2009,null,"""rel""","""f""","""65"""
"""Zimbabwe""","""ZW""","""ZWE""",2010,null,"""rel""","""f""","""65"""
"""Zimbabwe""","""ZW""","""ZWE""",2011,null,"""rel""","""f""","""65"""


In [44]:
cols = out_naive.columns
out_naive.sort(by=pl.all()).equals(out.select(cols).sort(by=pl.all()))

True

In [45]:
%%timeit
index = ["country", "iso2", "iso3", "year"]
columns = pl.Series(who.columns)
mask = columns.is_in(index).not_()
columns = columns.filter(mask)
regex = r"new_?(?<diagnosis>.*)_(?<gender>.)(?<age>.*)"
columns = columns.str.extract_groups(regex)
columns = columns.struct.unnest()
out = who.select(pl.implode("*"))
out = out.unpivot(index=index, value_name="count")
out = out.drop('variable')
out = out.with_columns(columns)
index.append('count')
out = out.explode(index)
out

5.61 ms ± 233 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


````{margin}
```{note}
Note that the primary performance culprit here is the regex computation - other string operations may offer better performance e.g string slice - in which case, you probably would not need the implode/explode approach.
```
````

That is a significant improvement in performance - about 20x faster.

By changing our approach and shifting the computation to a less expensive section, we claw back some performance gains. 



In [46]:
def wide_to_long_optimised(df):
    index = ["country", "iso2", "iso3", "year"]
    columns = pl.Series(df.columns)
    mask = columns.is_in(index).not_()
    columns = columns.filter(mask)
    regex = r"new_?(?<diagnosis>.*)_(?<gender>.)(?<age>.*)"
    columns = columns.str.extract_groups(regex)
    columns = columns.struct.unnest()
    out = df.select(pl.implode("*"))
    out = out.unpivot(index=index, value_name="count")
    out = out.drop("variable")
    out = out.with_columns(columns)
    index.append("count")
    out = out.explode(index)
    return out

In [47]:
%lprun -f wide_to_long_optimised wide_to_long_optimised(df=who)

Timer unit: 1e-09 s

Total time: 0.006154 s
File: /var/folders/yh/003fk4jn5fl9kl3mnm1y0b1h0000gr/T/ipykernel_83318/509568358.py
Function: wide_to_long_optimised at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def wide_to_long_optimised(df):
     2         1          0.0      0.0      0.0      index = ["country", "iso2", "iso3", "year"]
     3         1      34000.0  34000.0      0.6      columns = pl.Series(df.columns)
     4         1     223000.0 223000.0      3.6      mask = columns.is_in(index).not_()
     5         1       6000.0   6000.0      0.1      columns = columns.filter(mask)
     6         1          0.0      0.0      0.0      regex = r"new_?(?<diagnosis>.*)_(?<gender>.)(?<age>.*)"
     7         1     190000.0 190000.0      3.1      columns = columns.str.extract_groups(regex)
     8         1       6000.0   6000.0      0.1      columns = columns.struct.unnest()
     9         1     354000.0 354000.

```
Timer unit: 1e-09 s

Total time: 0.00589 s
File: /var/folders/yh/003fk4jn5fl9kl3mnm1y0b1h0000gr/T/ipykernel_83318/509568358.py
Function: wide_to_long_optimised at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def wide_to_long_optimised(df):
     2         1          0.0      0.0      0.0      index = ["country", "iso2", "iso3", "year"]
     3         1      29000.0  29000.0      0.5      columns = pl.Series(df.columns)
     4         1     232000.0 232000.0      3.9      mask = columns.is_in(index).not_()
     5         1       4000.0   4000.0      0.1      columns = columns.filter(mask)
     6         1          0.0      0.0      0.0      regex = r"new_?(?<diagnosis>.*)_(?<gender>.)(?<age>.*)"
     7         1     299000.0 299000.0      5.1      columns = columns.str.extract_groups(regex)
     8         1       5000.0   5000.0      0.1      columns = columns.struct.unnest()
     9         1     439000.0 439000.0      7.5      out = df.select(pl.implode("*"))
    10         1    3119000.0    3e+06     53.0      out = out.unpivot(index=index, value_name="count")
    11         1      34000.0  34000.0      0.6      out = out.drop("variable")
    12         1      60000.0  60000.0      1.0      out = out.with_columns(columns)
    13         1          0.0      0.0      0.0      index.append("count")
    14         1    1669000.0    2e+06     28.3      out = out.explode(index)
    15         1          0.0      0.0      0.0      return out
```

## **Multiple observations per row**

In [48]:
household = pl.read_csv("./Data_files/household.csv", null_values="NA")
household

family,dob_child1,dob_child2,name_child1,name_child2
i64,str,str,str,str
1,"""1998-11-26""","""2000-01-29""","""Susan""","""Jose"""
2,"""1996-06-22""",null,"""Mark""",null
3,"""2002-07-11""","""2004-04-05""","""Sam""","""Seth"""
4,"""2004-10-10""","""2009-08-27""","""Craig""","""Khai"""
5,"""2000-12-05""","""2005-02-28""","""Parker""","""Gracie"""


In the [previous blogpost](Reshape-Data-in-Polars-Wide-to_Long-Part-I.ipynb), we kept the `dob` and `name` as column headers, while flipping the remaining parts of the columns to rows: 

In [49]:
expression = pl.col("variable").str.split("_")
expression = expression.list.to_struct(fields=["header", "child"])
(
    household.unpivot(index="family")
    .drop_nulls("value")
    .with_columns(variable=expression)
    .unnest("variable")
    .pivot(index=["family", "child"], on="header", values="value")
)

family,child,dob,name
i64,str,str,str
1,"""child1""","""1998-11-26""","""Susan"""
2,"""child1""","""1996-06-22""","""Mark"""
3,"""child1""","""2002-07-11""","""Sam"""
4,"""child1""","""2004-10-10""","""Craig"""
5,"""child1""","""2000-12-05""","""Parker"""
1,"""child2""","""2000-01-29""","""Jose"""
3,"""child2""","""2004-04-05""","""Seth"""
4,"""child2""","""2009-08-27""","""Khai"""
5,"""child2""","""2005-02-28""","""Gracie"""


In [50]:
%%timeit
expression = pl.col("variable").str.split("_")
expression = expression.list.to_struct(fields=["header", "child"])
(
    household.unpivot(index="family")
    .drop_nulls("value")
    .with_columns(variable=expression)
    .unnest("variable")
    .pivot(index=["family", "child"], on="header", values="value")
)

497 μs ± 10.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Let's see the performance for a similar dataset, but larger:

In [51]:
from zipfile import ZipFile
zip_file = ZipFile("./Data_files/household_large.csv.zip")
zip_file = zip_file.read("household_large.csv")
household_large = pl.read_csv(zip_file, null_values="NA")
household_large

family,name_child0,dob_child0,name_child1,dob_child1,name_child2,dob_child2,name_child3,dob_child3,name_child4,dob_child4,name_child5,dob_child5,name_child6,dob_child6,name_child7,dob_child7,name_child8,dob_child8,name_child9,dob_child9,name_child10,dob_child10,name_child11,dob_child11,name_child12,dob_child12,name_child13,dob_child13,name_child14,dob_child14,name_child15,dob_child15,name_child16,dob_child16,name_child17,dob_child17,name_child18,dob_child18,name_child19,dob_child19,name_child20,dob_child20,name_child21,dob_child21,name_child22,dob_child22,name_child23,dob_child23,name_child24,dob_child24,name_child25,dob_child25,name_child26,dob_child26,name_child27,dob_child27,name_child28,dob_child28,name_child29,dob_child29
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
0,"""Brittany Mcdonald""","""1989-08-24""","""Karen Mendoza MD""","""1938-12-01""","""Stephanie Casey""","""1959-08-07""","""Rhonda Hartman""","""1984-01-17""","""Elizabeth Sampson""","""1947-01-25""","""Jacqueline Barrett""","""1949-02-15""","""Rebecca Harvey""","""1974-12-03""","""John Hopkins""","""2009-11-27""","""Diane Ali""","""1993-03-04""","""Emily Jones""","""1933-01-27""","""Rickey Bates""","""1938-06-03""","""Walter Stuart""","""1971-12-04""","""Megan Smith""","""1992-06-05""","""Amanda Morris""","""1976-02-01""","""Jennifer Greer""","""1971-12-12""","""Megan Haney""","""1935-09-11""","""Alyssa Garcia""","""1963-09-03""","""Daniel Turner""","""1938-11-17""","""Brittany Simmons""","""1919-11-21""","""Kurt Schwartz""","""1956-02-27""","""Tanya Brandt""","""1943-08-16""","""Jennifer White""","""1960-08-27""","""Eddie Hamilton""","""1988-01-15""","""Sean Chavez""","""1968-12-08""","""Rhonda White""","""1948-08-30""","""Linda Hicks""","""1936-08-03""","""Brittany Beard""","""1996-03-08""","""Nicole Schmidt""","""1948-12-06""","""Nancy Clark""","""1914-12-12""","""Dr. David Walters""","""1922-10-24"""
1,"""Diana Welch""","""1989-02-11""","""Brandon Pittman""","""2019-09-02""","""Alicia Smith""","""1923-06-27""","""Sarah Dean DVM""","""2001-02-10""","""Katherine Landry""","""1935-06-26""","""Sarah Hughes""","""1911-01-19""","""Amanda Peters""","""1922-06-02""","""Shawn Sanders""","""1936-11-17""","""David White""","""1998-02-25""","""Justin Mccormick""","""1994-03-06""","""James Kelly""","""2005-02-13""","""Kimberly Lynch MD""","""1986-02-18""","""Kimberly Hall""","""2001-07-23""","""Susan Anderson""","""1908-10-26""","""Anthony Hall""","""2012-10-10""","""Lisa Gardner""","""1939-08-17""","""Andrew Burke""","""1981-07-13""","""Brittney Evans""","""1993-06-06""","""Taylor Wood""","""1951-07-29""","""Tanya Taylor""","""2019-02-01""","""Ashley Schultz""","""1993-04-15""","""Amanda Sanchez""","""1923-07-14""","""Harry Stevens""","""1986-11-10""","""David Flores""","""1931-09-16""","""Timothy Gray""","""2017-05-07""","""Jay Ramsey""","""1936-05-08""","""Charles Bell""","""2014-12-05""","""Jacob Williams""","""1963-04-17""","""Kimberly Gonzalez""","""1991-11-15""","""Monica Newton""","""1923-10-27"""
2,"""Jamie Richards""","""1973-11-04""","""Kristin Dunn""","""1973-04-02""","""Kathy Vasquez""","""1966-03-20""","""William Johnson""","""2006-03-08""","""Anthony Brewer""","""1987-06-21""","""James Hendricks""","""1981-03-29""","""Emily Torres""","""1990-11-09""","""Jennifer Santos""","""1982-02-13""","""Travis Snyder""","""2018-09-04""","""Misty Hudson""","""1974-01-03""","""Mathew Price""","""1965-07-09""","""Charles Anthony""","""2003-07-20""","""Sean Cook""","""1919-03-26""","""Krista Hall""","""1951-02-12""","""Sarah Hurst""","""1962-09-22""","""Angela May""","""2009-11-07""","""Antonio Ayers""","""1980-12-11""","""Kathryn Beck""","""1944-05-27""","""Barbara Davis""","""1929-04-19""","""Vanessa Gonzalez""","""1944-06-13""","""Tina Benjamin""","""1911-02-01""","""An

In [52]:
household_large.shape

(100000, 61)

In [53]:
expression = pl.col("variable").str.split("_")
expression = expression.list.to_struct(fields=["header", "child"])
out_naive = (
    household_large.unpivot(index="family")
    .drop_nulls("value")
    .with_columns(variable=expression)
    .unnest("variable")
    .pivot(index=["family", "child"], on="header", values="value")
)
out_naive

family,child,name,dob
i64,str,str,str
0,"""child0""","""Brittany Mcdonald""","""1989-08-24"""
1,"""child0""","""Diana Welch""","""1989-02-11"""
2,"""child0""","""Jamie Richards""","""1973-11-04"""
3,"""child0""","""Michele Reed""","""1994-11-05"""
4,"""child0""","""Steven Burgess""","""1997-12-24"""
…,…,…,…
99995,"""child29""","""Patrick Solomon""","""1920-03-12"""
99996,"""child29""","""Steven Smith""","""1987-09-22"""
99997,"""child29""","""Scott Carlson""","""1950-05-23"""


In [54]:
%%timeit
expression = pl.col("variable").str.split("_")
expression = expression.list.to_struct(fields=["header", "child"])
(
    household_large.unpivot(index="family")
    .drop_nulls("value")
    .with_columns(variable=expression)
    .unnest("variable")
    .pivot(index=["family", "child"], on="header", values="value")
)

1.23 s ± 6.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
def lreshape(df):
    out = df.unpivot(index="family")
    out = out.drop_nulls("value")
    string = out.get_column("variable")
    string = string.str.split("_")
    string = string.list.to_struct(fields=["header", "child"])
    out = out.with_columns(variable=string)
    out = out.unnest("variable")
    out = out.pivot(index=["family", "child"], on="header", values="value")
    return out

In [56]:
%lprun -f lreshape lreshape(household_large)

Timer unit: 1e-09 s

Total time: 1.24179 s
File: /var/folders/yh/003fk4jn5fl9kl3mnm1y0b1h0000gr/T/ipykernel_83318/2283907795.py
Function: lreshape at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def lreshape(df):
     2         1   56682000.0    6e+07      4.6      out = df.unpivot(index="family")
     3         1     242000.0 242000.0      0.0      out = out.drop_nulls("value")
     4         1       6000.0   6000.0      0.0      string = out.get_column("variable")
     5         1  232017000.0    2e+08     18.7      string = string.str.split("_")
     6         1   42616000.0    4e+07      3.4      string = string.list.to_struct(fields=["header", "child"])
     7         1    1193000.0    1e+06      0.1      out = out.with_columns(variable=string)
     8         1      21000.0  21000.0      0.0      out = out.unnest("variable")
     9         1  909016000.0    9e+08     73.2      out = out.pivot(index=["family

```
Timer unit: 1e-09 s

Total time: 1.14191 s
File: /var/folders/yh/003fk4jn5fl9kl3mnm1y0b1h0000gr/T/ipykernel_83318/2283907795.py
Function: lreshape at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def lreshape(df):
     2         1   55695000.0    6e+07      4.9      out = df.unpivot(index="family")
     3         1     268000.0 268000.0      0.0      out = out.drop_nulls("value")
     4         1       9000.0   9000.0      0.0      string = out.get_column("variable")
     5         1  222072000.0    2e+08     19.4      string = string.str.split("_")
     6         1   31940000.0    3e+07      2.8      string = string.list.to_struct(fields=["header", "child"])
     7         1    1298000.0    1e+06      0.1      out = out.with_columns(variable=string)
     8         1      17000.0  17000.0      0.0      out = out.unnest("variable")
     9         1  830615000.0    8e+08     72.7      out = out.pivot(index=["family", "child"], on="header", values="value")
    10         1          0.0      0.0      0.0      return out
```

The `pivot` operation takes more than half of the reshaping; looks like a good place to start, with a view to improving performance, if possible. 

To possibly improve performance, we need to find a way to pair the headers(`dob`, `name`) with the relevant columns, before unpivoting. 

We will also process the columns before unpivoting, based on what we learnt from our [previous reshaping exercise](#Many-variables-in-column-names).

Let's start with processing the columns :

In [57]:
index = "family"
columns = pl.Series("header", household_large.columns)
mask = columns.is_in([index]).not_()
columns = columns.filter(mask)
columns_ = columns.str.split("_")
columns_ = columns_.list.to_struct(fields=["header", "child"])
columns_ = columns_.struct.unnest()
columns_ = columns_.with_columns(columns=columns)
columns_

header,child,columns
str,str,str
"""name""","""child0""","""name_child0"""
"""dob""","""child0""","""dob_child0"""
"""name""","""child1""","""name_child1"""
"""dob""","""child1""","""dob_child1"""
"""name""","""child2""","""name_child2"""
…,…,…
"""dob""","""child27""","""dob_child27"""
"""name""","""child28""","""name_child28"""
"""dob""","""child28""","""dob_child28"""


The next step is to pair the `header` and `columns`, based on `child`:

In [58]:
# pass maintain_order=True
# if you want to retain the order
grouped = columns_.group_by("child") 
grouped = grouped.agg(pl.all())
grouped

child,header,columns
str,list[str],list[str]
"""child20""","[""name"", ""dob""]","[""name_child20"", ""dob_child20""]"
"""child18""","[""name"", ""dob""]","[""name_child18"", ""dob_child18""]"
"""child3""","[""name"", ""dob""]","[""name_child3"", ""dob_child3""]"
"""child26""","[""name"", ""dob""]","[""name_child26"", ""dob_child26""]"
"""child23""","[""name"", ""dob""]","[""name_child23"", ""dob_child23""]"
…,…,…
"""child5""","[""name"", ""dob""]","[""name_child5"", ""dob_child5""]"
"""child14""","[""name"", ""dob""]","[""name_child14"", ""dob_child14""]"
"""child27""","[""name"", ""dob""]","[""name_child27"", ""dob_child27""]"


From the above we can see a relationship; we'll create structs of pairs, where the field names are the headers(`dob`, `name`) and the values in the structs are the relevant column names:

In [59]:
expression = [index]
zipped = zip(
    grouped.get_column("header").to_list(),
    grouped.get_column("columns").to_list(),
)
for num, (heads, column_names) in enumerate(zipped):
    expr_ = dict(zip(heads, column_names))
    expr_ = pl.struct(**expr_).alias(str(num))
    expression.append(expr_)

Now, we unpivot:

In [60]:
out = household_large.select(expression)
out = out.select(pl.implode('*'))
out = out.unpivot(index=index)
out = out.with_columns(grouped.get_column('child'))
out = out.drop('variable')
out = out.explode(['family','value'])
out = out.unnest('value')
out

family,name,dob,child
i64,str,str,str
0,"""Tanya Brandt""","""1943-08-16""","""child20"""
1,"""Ashley Schultz""","""1993-04-15""","""child20"""
2,"""Tina Benjamin""","""1911-02-01""","""child20"""
3,"""Eric Santos""","""1928-02-18""","""child20"""
4,"""Roger Meza""","""1934-12-10""","""child20"""
…,…,…,…
99995,"""Jennifer Flores""","""1933-06-17""","""child16"""
99996,"""Amanda Ruiz""","""2018-08-09""","""child16"""
99997,"""Angela Johnson""","""2015-06-24""","""child16"""


In [61]:
cols = out_naive.columns
out_naive.sort(by=pl.all()).equals(out.select(cols).sort(by=pl.all()))

True

In [62]:
%%timeit
index = "family"
columns = pl.Series("header", household_large.columns)
mask = columns.is_in([index]).not_()
columns = columns.filter(mask)
columns_ = columns.str.split("_")
columns_ = columns_.list.to_struct(fields=["header", "child"])
columns_ = columns_.struct.unnest()
columns_ = columns_.with_columns(columns=columns)
grouped = columns_.group_by("child")
grouped = grouped.agg(pl.all())
expression = [index]
zipped = zip(
    grouped.get_column("header").to_list(),
    grouped.get_column("columns").to_list(),
)
for num, (heads, column_names) in enumerate(zipped):
    expr_ = dict(zip(heads, column_names))
    expr_ = pl.struct(**expr_).alias(str(num))
    expression.append(expr_)
out = household_large.select(expression)
out = out.select(pl.implode('*'))
out = out.unpivot(index=index)
out = out.with_columns(grouped.get_column('child'))
out = out.drop('variable')
out = out.explode(['family','value'])
out = out.unnest('value')
out

83.1 ms ± 190 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


By changing our approach (preprocess the columns, and grouping related columns into structs before unpivoting) we improve the peformance significantly - about 14x faster.

In [63]:
def lreshape_optimised(df):
    index = "family"
    columns = pl.Series("header", df.columns)
    mask = columns.is_in([index]).not_()
    columns = columns.filter(mask)
    columns_ = columns.str.split("_")
    columns_ = columns_.list.to_struct(fields=["header", "child"])
    columns_ = columns_.struct.unnest()
    columns_ = columns_.with_columns(columns=columns)
    grouped = columns_.group_by("child")
    grouped = grouped.agg(pl.all())
    expression = [index]
    zipped = zip(
        grouped.get_column("header").to_list(),
        grouped.get_column("columns").to_list(),
    )
    for num, (heads, column_names) in enumerate(zipped):
        expr_ = dict(zip(heads, column_names))
        expr_ = pl.struct(**expr_).alias(str(num))
        expression.append(expr_)
    out = df.select(expression)
    out = out.select(pl.implode('*'))
    out = out.unpivot(index=index)
    out = out.with_columns(grouped.get_column('child'))
    out = out.drop('variable')
    out = out.explode(['family','value'])
    out = out.unnest('value')
    return out

In [64]:
%lprun -f lreshape_optimised lreshape_optimised(household_large)

Timer unit: 1e-09 s

Total time: 0.083045 s
File: /var/folders/yh/003fk4jn5fl9kl3mnm1y0b1h0000gr/T/ipykernel_83318/2813683113.py
Function: lreshape_optimised at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def lreshape_optimised(df):
     2         1       1000.0   1000.0      0.0      index = "family"
     3         1      43000.0  43000.0      0.1      columns = pl.Series("header", df.columns)
     4         1     278000.0 278000.0      0.3      mask = columns.is_in([index]).not_()
     5         1      12000.0  12000.0      0.0      columns = columns.filter(mask)
     6         1     196000.0 196000.0      0.2      columns_ = columns.str.split("_")
     7         1     244000.0 244000.0      0.3      columns_ = columns_.list.to_struct(fields=["header", "child"])
     8         1       6000.0   6000.0      0.0      columns_ = columns_.struct.unnest()
     9         1     127000.0 127000.0      0.2      columns

```
Timer unit: 1e-09 s

Total time: 0.08356 s
File: /var/folders/yh/003fk4jn5fl9kl3mnm1y0b1h0000gr/T/ipykernel_83318/2813683113.py
Function: lreshape_optimised at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def lreshape_optimised(df):
     2         1          0.0      0.0      0.0      index = "family"
     3         1      34000.0  34000.0      0.0      columns = pl.Series("header", df.columns)
     4         1     165000.0 165000.0      0.2      mask = columns.is_in([index]).not_()
     5         1       6000.0   6000.0      0.0      columns = columns.filter(mask)
     6         1     136000.0 136000.0      0.2      columns_ = columns.str.split("_")
     7         1     157000.0 157000.0      0.2      columns_ = columns_.list.to_struct(fields=["header", "child"])
     8         1       8000.0   8000.0      0.0      columns_ = columns_.struct.unnest()
     9         1     187000.0 187000.0      0.2      columns_ = columns_.with_columns(columns=columns)
    10         1       4000.0   4000.0      0.0      grouped = columns_.group_by("child")
    11         1     274000.0 274000.0      0.3      grouped = grouped.agg(pl.all())
    12         1       1000.0   1000.0      0.0      expression = [index]
    13         2       1000.0    500.0      0.0      zipped = zip(
    14         1      14000.0  14000.0      0.0          grouped.get_column("header").to_list(),
    15         1       6000.0   6000.0      0.0          grouped.get_column("columns").to_list(),
    16                                               )
    17        31       8000.0    258.1      0.0      for num, (heads, column_names) in enumerate(zipped):
    18        30      12000.0    400.0      0.0          expr_ = dict(zip(heads, column_names))
    19        30     193000.0   6433.3      0.2          expr_ = pl.struct(**expr_).alias(str(num))
    20        30       5000.0    166.7      0.0          expression.append(expr_)
    21         1     719000.0 719000.0      0.9      out = df.select(expression)
    22         1    6927000.0    7e+06      8.3      out = out.select(pl.implode('*'))
    23         1   57090000.0    6e+07     68.3      out = out.unpivot(index=index)
    24         1      93000.0  93000.0      0.1      out = out.with_columns(grouped.get_column('child'))
    25         1      23000.0  23000.0      0.0      out = out.drop('variable')
    26         1   17479000.0    2e+07     20.9      out = out.explode(['family','value'])
    27         1      18000.0  18000.0      0.0      out = out.unnest('value')
    28         1          0.0      0.0      0.0      return out
```

## **Summary**

This blog post highlights how to flip from wide to long form efficiently. 

One thing to note is the verbosity of the faster approach. In another blog post, we'll have a look at two functions from the [pyjanitor](https://pyjanitor-devs.github.io/pyjanitor/) library that offer conciseness and efficiency when reshaping from wide to long form.